# Properly Ordered Card Hands

The 538 Riddler [presented](https://fivethirtyeight.com/features/who-will-capture-the-most-james-bonds/) this problem by Matt Ginsberg:
    
> *You play so many card games that you’ve developed a very specific organizational obsession. When you’re dealt your hand, you want to organize it such that the cards of a given suit are grouped together and, if possible, such that no suited groups of the same color are adjacent. (Numbers don’t matter to you.) Moreover, when you receive your randomly ordered hand, you want to achieve this organization with a single motion, moving only one adjacent block of cards to some other position in your hand, maintaining the original order of that block and other cards, except for that one move.*

> *Suppose you’re playing pitch, in which a hand has six cards. What are the odds that you can accomplish your obsessive goal? What about for another game, where a hand has N cards, somewhere between 1 and 13?*

The first thing to decide is how many `N`-card hands are there? That will tell if I can just use brute force, looking at every possible hand. The answer is (52 choose `N`), so we have:

- 6 cards: 20,358,520 hands
- 13 cards: 635,013,559,600 hands 

That's too many hands. 

# Abstract Hands

I notice that the problem states *"Numbers don’t matter,"* so I can abstract away from *cards* to *suits*: instead of saying that the first card in this hand is the seven of spades, I can just say it is a spade. Then there are only 4<sup>N</sup> abstract hands (for N &le; 13), so we have:

- 6 cards: 4,096 abstract hands
- 13 cards: 67,108,864 abstract hands

That's a big improvement.  

# Deals: Hands and their Probabilities

There are two red suits and two black suits, so I'll represent the four suits with the characters `'rbRB'`. (I also considered using `'♠️♥️♦️♣️'`.) I'll represent an abstract hand as a string of suits: `'rrBrbr'` is a 6-card hand. I'll define `deals(N)` to return a dict of all possible abstract hands of length `N`, each mapped to the probability of the hand. 

With actual hands, every hand has the same probability, because every card is equally likely to be the next card dealt. But with abstract hands, the probability of the next suit depends on how many cards of that suit have already been dealt. If I've already dealt the 12 cards `'rrrrrrrrrrrr'`, then the probability of the next card being an `'r'` is 1/40, and the probability of it being a `'b'` is 13/40. So as I build up the abstract hands, I'll need to keep track of the number of remaining cards of each suit.

I'll use `Fraction` to get exact arithmetic and `lru_cache` to avoid repeated computations.

In [1]:
import re
from fractions import Fraction
from functools import lru_cache

one   = Fraction(1)
suits = 'rbRB'

@lru_cache()
def deals(N):
    "A dict of {hand: probability} for all hands of length N."
    if N == 0:
        return {'': one}
    else:
        return {hand + suit: p * (13 - hand.count(suit)) / (52 - len(hand))
                for (hand, p) in deals(N - 1).items()
                for suit in suits}

In [2]:
deals(2)

{'BB': Fraction(1, 17),
 'BR': Fraction(13, 204),
 'Bb': Fraction(13, 204),
 'Br': Fraction(13, 204),
 'RB': Fraction(13, 204),
 'RR': Fraction(1, 17),
 'Rb': Fraction(13, 204),
 'Rr': Fraction(13, 204),
 'bB': Fraction(13, 204),
 'bR': Fraction(13, 204),
 'bb': Fraction(1, 17),
 'br': Fraction(13, 204),
 'rB': Fraction(13, 204),
 'rR': Fraction(13, 204),
 'rb': Fraction(13, 204),
 'rr': Fraction(1, 17)}

Is that right? Yes it is. The probability of `'BB'` is 1/17, beause the probability of the first `'B'` is 13/52 or 1/4, and when we deal the second card, one `'B'` is gone, so the probability is 12/51, so that simplifies to 1/4 &times; 12/51 = 3/51 = 1/17. The probability of `'BR'` is 13/204, because the probability of the `'R'` is 13/51, and 1/4 &times; 13/51 = 13/204.

# Collapsing Hands

I'll introduce the idea of *collapsing* an abstract hand by replacing a run of cards of the same suit with a single card, so that: 

     collapse('BBBBBrrrrBBBB') == 'BrB'
     
From now on I'll use the term *hand* rather than *abstract hand* for things like `'BBBBBrrrrBBBB'`, and I'll use the terms *sequence* or *seq* for the collapsed version, `'BrB'`.

# Properly Ordered Hands

A hand is considered properly `ordered` if *"the cards of a given suit are grouped together and, if possible, such that no suited groups of the same color are adjacent."* I was initially confused about the meaning of *"if possible";* Matt Ginsberg confirmed it means *"if it is possible to separate the colors in any number of moves"*, and thus that the hand `'BBBbbb'` is properly ordered, because it is not possible to separate the two black suits, while `'BBBbbR'` is not properly ordered, because the red card could have been inserted between the two black runs.

So a hand is properly ordered if, considering its collapsed sequence, each suit appears only once, and either all the colors are the same, or suits of the same color don't appear adjacent to each other.

In [3]:
def ordered(hand):
    "Properly ordered if each suit run appears only once, and same color suits not adjacent."
    seq = collapse(hand)
    return once_each(seq) and (len(colors(seq)) == 1 or not adjacent_colors(seq))
                                 
def collapse(hand):
    "Collapse identical adjacent characters to one character."
    return ''.join(hand[i] for i in range(len(hand)) if i == 0 or hand[i] != hand[i-1])

def once_each(seq): return len(seq) == len(set(seq))
def colors(seq):    return set(seq.casefold())
adjacent_colors =   re.compile('rR|Rr|Bb|bB').search

In [4]:
collapse('BBBBBrrrrBBBB')

'BrB'

In [5]:
ordered('BBBbbR') 

False

# Moving Cards to Make a Hand Ordered

I won't try to be clever; I'm content to use brute force. I'll say that a collapsed sequence is `orderable` if any of the possible `moves` of a block of cards makes the hand `ordered`. I'll find all possible `moves`, by finding all possible  `splits` of the cards into a middle block of cards flanked by (possibly empty) left and right sequences; then all possible `inserts` of the block back into the rest of the cards.  I'll define `orderable_probability(N)` to give the probability that a random `N`-card hand is orderable.
Since many hands will collapse to the same sequence, I'll throw a `lru_cache` onto `orderable` so that it won't have to repeat computations.

In [6]:
@lru_cache(None)
def orderable(seq): 
    "Can this collapsed sequence be put into proper order in one move?"
    return any(ordered(m) for m in moves(seq))

def orderable_probability(N):
    "What's the probability that an N-card hand is orderable?"
    return sum(p for (hand, p) in deals(N).items() if orderable(collapse(hand)))

def moves(seq):
    "All possible ways of moving a single block of cards."
    return {collapse(s) for (L, block, R) in splits(seq)
            for s in inserts(block, L + R)}

def inserts(block, others):
    "All ways of inserting a block into the other cards."
    return [others[:i] + block + others[i:]
            for i in range(len(others) + 1)]

def splits(seq):
    "All ways of splitting a hand into a non-empty block flanked by left and right parts."
    return [(seq[:i], seq[i:j], seq[j:])
            for i in range(len(seq))
            for j in range(i + 1, len(seq) + 1)]

# First Answer

Here's our answer for 6 cards:

In [7]:
orderable_probability(6)

Fraction(51083, 83895)

And an easier-to-read answer for everthing up to `N` = 6 cards:

In [8]:
def report(Ns):
    "Show the probability of orderability, for each N in Ns."
    for N in Ns:
        P = orderable_probability(N)
        print('{:2}: {:6.1%} = {}'.format(N, float(P), P))
        
report(range(7))

 0:   0.0% = 0
 1: 100.0% = 1
 2: 100.0% = 1
 3: 100.0% = 1
 4: 100.0% = 1
 5:  85.2% = 213019/249900
 6:  60.9% = 51083/83895


# Getting to `N` = 13

That looks good, but if we want to get to 13-card hands, we would have to handle 4<sup>13</sup> = 67,108,864 `deals`, which would take a while. I can speed things up by taking advantage of two key properties of orderable sequences:

1. **An orderable sequence can have at most 7 runs.** We know that a properly ordered hand can have at most 4 runs. But a move can reduce the number of runs by only 3 at the most:  one run can be reduced when we remove the block (if the cards on either side of the block are the same), and two more can be reduced when we replace the block  (if the left and right ends of the block match the suits to the left and right of the new position). Here's an example of moving a block [bracketed] to reduce the number of runs from 6 to 3:

       bRB[bR]B   =>   b[bR]RBB  =   bRB
       
2. **Adding a suit to the end of an unorderable sequence can't make it orderable.** Even after we move a block, you can't make an unordered hand ordered by inserting one suit.


I'll redefine `deals(N)` to hold only orderable hands, redefine `orderable(seq)` to immediately reject sequences longer than 7, and redefine `orderable_probability(N)` to just add up the probabalities in `deals(N)`:

In [9]:
@lru_cache()
def deals(N):
    "A dict of {hand: probability} for all orderable hands of length N."
    if N == 0:
        return {'': one}
    else:
        return {hand + suit: p * (13 - hand.count(suit)) / (52 - len(hand))
                for (hand, p) in deals(N - 1).items()
                for suit in suits
                if orderable(collapse(hand + suit))}
    
@lru_cache(None)
def orderable(seq): 
    "Can this collapsed sequence be put into proper order in one move?"
    return len(seq) <= 7 and any(ordered(m) for m in moves(seq))

def orderable_probability(N):
    "What's the probability that an N-card hand is orderable?"
    return sum(deals(N).values())

# Final Answer

We're finaly ready to go up to `N` = 13:

In [10]:
%time report(range(14))

 0: 100.0% = 1
 1: 100.0% = 1
 2: 100.0% = 1
 3: 100.0% = 1
 4: 100.0% = 1
 5:  85.2% = 213019/249900
 6:  60.9% = 51083/83895
 7:  37.3% = 33606799/90047300
 8:  20.2% = 29210911/144718875
 9:   9.9% = 133194539/1350709500
10:   4.4% = 367755247/8297215500
11:   1.9% = 22673450197/1219690678500
12:   0.7% = 1751664923/238130084850
13:   0.3% = 30785713171/11112737293000
CPU times: user 16.6 s, sys: 337 ms, total: 17 s
Wall time: 19.3 s


# Cache Sizes

Let's look at the cache for  `orderable(seq)`:

In [11]:
orderable.cache_info()

CacheInfo(hits=1438512, misses=1540, maxsize=None, currsize=1540)

So we looked at over 7 million hands, but only 4373 different collapsed sequences. And once we hit `N` = 7, we've seen all the sequences we're ever going to see. From `N` = 8 and up, almost all the computation goes into computing the probability of each hand, and collapsing the hand into a sequence, not into deciding the orderability of each sequence.

We save a lot of space in the `deals(N)` caches. Instead of storing all 4<sup>13</sup> hands for `deals(13)`, the output above says that just 0.3% of the hands are orderable, so we reduced the cache size by a factor of 300 or so.

# Unit Tests

To gain confidence in these answers, here are some unit tests. Before declaring my answers definitively correct, I would want a lot more tests, and some independent code reviews.

In [12]:
def test():
    assert deals(1) == {'B': 1/4, 'R': 1/4, 'b': 1/4, 'r': 1/4}
    assert ordered('BBBBBrrrrBBBB') is False
    assert ordered('BBBBBrrrrRRRR') is False
    assert ordered('BBBbbr') is False # Bb
    assert ordered('BBBbbrB') is False # two B's
    assert ordered('BBBbbb') 
    assert ordered('BBBbbbB') is False # two B's
    assert ordered('BBBBBrrrrbbbb')
    assert colors('BBBBBrrrrbbbb') == {'r', 'b'}
    assert once_each('Bb')
    assert once_each('BbR')
    assert adjacent_colors('BBBbbR')
    assert not adjacent_colors('BBBBBrrrrBBBB')
    assert collapse('BBBBBrrrrBBBB') == 'BrB'
    assert collapse('brBBrrRR') == 'brBrR'
    assert collapse('bbbbBBBrrr') == 'bBr'
    assert moves('bRb') == {'Rb', 'bR', 'bRb'}
    assert moves('bRBb') == {
        'BbR', 'BbRb', 'RBb', 'RbBb', 'bBRb', 'bBbR', 'bRB', 'bRBb', 'bRbB'}
    assert inserts('BB', '....') == [
        'BB....', '.BB...', '..BB..', '...BB.', '....BB']
    assert splits('123') == [('', '1', '23'), ('', '12', '3'), ('', '123', ''),
                             ('1', '2', '3'), ('1', '23', ''), ('12', '3', '')]
    assert orderable('bBr') # move 'r' after 'b'
    assert orderable('bBrbRBr') # move 'bRB' after first 'b' to get 'bbRBBrr'
    assert orderable('bBrbRBrb') is False
    return True

test()

True